## Lesson Notebook 6: Machine Translation

In this notebook we will look at several examples related to machine translation:

   * Simple translation examples with T5 

   * Translation example with M2M100 - many more languages

   * MT metrics examples

   * Subword models and tokenizers

   * Transformer encoder decoder translation of Shakespeare to Modern English

Part 6 reuses the transformer code from the Keras Tutorial https://keras.io/examples/nlp/neural_machine_translation_with_keras_nlp/


<a id = 'returnToTop'></a>

## Notebook Contents
  * 1. [Setup](#setup)
  * 2. [Simple Translation Model](#simpleTranslation)
  * 3. [M2M100 Translation Example](#m2mTranslation)
  * 4. [Machine Translation Metrics](#translationMetrics)  
  * 5. [Subword Models](#subwordModels)
  * 6. [Shakespeare-to-Modern English translation with a sequence-to-sequence Transformer](#shakespeare)
  * [Answers](#answers)







  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2023-spring-main/blob/master/materials/lesson_notebooks/lesson_6_Machine_Translation.ipynb)

[Return to Top](#returnToTop)  
<a id = 'setup'></a>

## 1. Setup


We'll start with the usual setup. We need to begin with the sentencepiece code in order to tokenize the text for some of the models.

In [1]:
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00


In [2]:
!pip install -q transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.3 MB/s eta 0:00:00


In [3]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.8 MB/s eta 0:00:00


In [4]:
!pip install -q git+https://github.com/keras-team/keras-nlp.git --upgrade

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.1 MB/s eta 0:00:00


In [5]:
#Am I running a GPU and what type is it?
!nvidia-smi

Mon Feb 13 05:09:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W /  70W |      0MiB / 15360MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[Return to Top](#returnToTop)  
<a id = 'simpleTranslation'></a>


## 2. Simple Translation Example

These T5 models are trained to translate in one direction only.  For example, they can translate from English to French but not from French to English. 

Let's test this out.

In [6]:
#import T5 and show 
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration

In [7]:
SENTENCE_TO_TRANSLATE = ( "PG&E stated it scheduled the blackouts in response to forecasts for high winds \
            amid dry conditions.")

BACK_TRANSLATE_TEST = ("PG&E a déclaré qu'elle avait prévu les panne de courant.")

In [8]:
t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-base') #also t5-small and t5-large
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

t5_model.summary()

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222,903,552
Trainable params: 222,903,552
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Add the prompt to the sentence we want to translate so the model knows what we want it to do with the input.

In [9]:
t5_input_text = "translate english to french: " + SENTENCE_TO_TRANSLATE

In [10]:
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')

**QUESTION 1**: What do the inputs look like?  We've already seen BERT inputs. What's happening with T5? What's the same as what we saw with BERT and what's different? 

In [11]:
t5_inputs

{'input_ids': <tf.Tensor: shape=(1, 29), dtype=int32, numpy=
array([[13959, 22269,    12, 20609,    10,     3,  7861,   184,   427,
         4568,    34,  5018,     8,  1001,   670,     7,    16,  1773,
           12,  7555,     7,    21,   306, 13551, 18905,  2192,  1124,
            5,     1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 29), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [12]:
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                   max_length=20)
                             
print([t5_tokenizer.decode(g, skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False) for g in t5_summary_ids])

["PG&E a déclaré qu'elle avait prévu les panne de courant"]


Not bad. Now let's try the reverse even though we know the model wasn't trained to translate in that direction.  What do you think it will do?

In [13]:
t5_back_text = "translate french to english: " + BACK_TRANSLATE_TEST

In [14]:
t5_binputs = t5_tokenizer([t5_back_text], return_tensors='tf')

The decoder still runs and emits language, specifically French, as requested.  These models will pretty much always produce some output but you need to make sure that you're asking it to do something it can and that it is doing the right thing.

In [15]:
t5_summary_ids = t5_model.generate(t5_binputs['input_ids'],
                                   max_length=20)
                             
print([t5_tokenizer.decode(g, skip_special_tokens=True, 
                           clean_up_tokenization_spaces=False) for g in t5_summary_ids])

["PG&E a déclaré qu'elle avait prévu les panne de courant"]


[Return to Top](#returnToTop)  
<a id = 'm2mTranslation'></a>


## 3. M2M100 Translation Example

M2M100 is a large model that was pre-trained on many languages simultaneoulsy.  You do need to give it some clues about what you are expecting when it translates.  Typically this takes the form of specifying the input and the output languages.  Let's look at the tokenizer first. How can it handle 100 different languages?

In [16]:
from transformers import M2M100Config, M2M100Tokenizer

tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="en", tgt_lang="fr")

tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['▁Don',
 "'",
 't',
 '▁you',
 '▁love',
 '▁',
 '🤗',
 '▁Transform',
 'ers',
 '?',
 '▁We',
 '▁sure',
 '▁do',
 '.']

Now let's try to translate.  We have two original sentences, one in English and one in Chinese that have roughly the same meaning.

In [17]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

en_text = "Do not meddle in the affairs of wizards, for they are subtle and quick to anger."
chinese_text = "不要插手巫師的事務, 因為他們是微妙的, 很快就會發怒."

tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M", src_lang="zh")
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")

In [18]:
encoded_zh = tokenizer(chinese_text, return_tensors="pt")

Let's start by taking the Chinese sentence and translate it back in to English.

In [19]:
generated_tokens = model.generate(**encoded_zh, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Do not interfere with the matters of the witches, because they are delicate and will soon be angry.']

Interesting and subtly different from our English original. Now we'll try translating the English to Chinese and then we'll take that Chinese output and translate it back into English.  This should give us an idea of how well the model works.

In [20]:
encoded_en = tokenizer(en_text, return_tensors="pt")

In [21]:
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("zh"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['不要介入魔術師的事情,因為他們是微妙和快樂的憤怒。']

Now we'll store that Chinese output in a variable so we can translate back to English.

In [22]:
chinese_back_text = '不要介入魔術師的事情,因為他們是微妙和快樂的憤怒。'

In [23]:
encoded_zhb = tokenizer(chinese_back_text, return_tensors="pt")

In [24]:
generated_tokens = model.generate(**encoded_zhb, forced_bos_token_id=tokenizer.get_lang_id("en"))
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['Do not interfere with the things of the witches, because they are delicate and pleasant anger.']

Now you can see how far it has drifted as we have translated back and forth. With some care this approach can be used to generate novel content that can augment a training set (as long as the drift isn't too bad). This is what we call back translation.

[Return to Top](#returnToTop)  
<a id = 'translationMetrics'></a>

## 4. Machine Translation Metrics

HuggingFace provides a library called evaluate that includes a large number of metrics.  We'll use two of them here.

In [25]:
!pip install -q evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.1 MB/s eta 0:00:00


### 4.1 BLEU example

The [BLEU metric](https://huggingface.co/spaces/evaluate-metric/bleu) has been around for awhile. Let's run an example of the scoring using the function provided by the evaluate library from HuggingFace.

In [26]:
#let's manually create some candidates and references
#individual sentece example - this is best to experiment with.
bleu_candidate = ["the earth trembled in Japan again on Monday the 4th of September"
]

bleu_reference = [["earthquakes hit Japan again on Monday September 4"]
]

#multiple pairs of inputs and reference outputs 
bleu_candidates = ["the earth trembled in Japan again on Monday the 4th of September", 
                   "earthquakes struck Japan again on Monday the 4th of September"
]
bleu_references = [
                   ["earthquakes hit Japan again on Monday September 4"],
                   ["On September 4th , a Monday , Japan had another earthquake"]
]

Let's first try our individual candidate and reference.  They're both sort of saying the same thing.  Does the BLEU score reflect that similarity?

In [27]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=bleu_candidate, references=bleu_reference)
print(results)

{'bleu': 0.22416933501922287, 'precisions': [0.4166666666666667, 0.2727272727272727, 0.2, 0.1111111111111111], 'brevity_penalty': 1.0, 'length_ratio': 1.5, 'translation_length': 12, 'reference_length': 8}


BLEU is typically used in aggregate with multiple candidates as well as multiple reference examples for each sentence pair.  Here we run the candidates and references so you can see how it's done. 

In [28]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=bleu_candidates, references=bleu_references)
print(results)

{'bleu': 0.14367696612929734, 'precisions': [0.4090909090909091, 0.15, 0.1111111111111111, 0.0625], 'brevity_penalty': 1.0, 'length_ratio': 1.1578947368421053, 'translation_length': 22, 'reference_length': 19}


### 4.2 BLEURT

Now let's try the [BLEURT metric](https://huggingface.co/spaces/evaluate-metric/bleurt).  It should score better than BLEU.  Again, think in terms of faithfulness and fluency.  How well does the candidate reflect the information in the reference? How well does the candidate read?

The [BLEURT model](https://aclanthology.org/2020.acl-main.704.pdf) is pretrained to evaluate text generation tasks.  We can use it in this pre-trained state or we can fine-tune it provided we have sentence pairs that are scored because BLEURT is learning to predict how a human would score the output.

In [29]:
!pip install -q git+https://github.com/google-research/bleurt.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 7.2 MB/s eta 0:00:00


In [30]:
bleurt = evaluate.load("bleurt", 'bleurt-base-512')

In [31]:
#let's manually create some candidates and references

candidates = ["the earth trembled in Japan again on Monday, the 4th of September"]
candidates2 = ["earthquakes shook Japan on Monday, the 4th of September"]
references = ["earthquakes hit Japan again on Monday, September 4"]

#Here are some other candidates you can experiment with
#candidates = ["the earth trembled in Japan again on Monday, the 4th of September", 
#              "earthquakes struck Japan again on Monday, the 4th of September",
#              "On September 4th, a Monday, Japan had another earthquake"]

In [32]:
results = bleurt.compute(predictions=candidates, references=references)
print(results)

{'scores': [-0.18725772202014923]}


Notice that the candidate and reference don't get a very good score. Do you think they should get a higher score?

Try running it again except this time use candidates2 and see what kind of score it gets.  Is that what you expected?

[Return to Top](#returnToTop)  
<a id = 'subwordModels'></a>

## 5. Subword Models

Different pretrained models use different subword models.  Each subword model identifies a different set of "tokens" based on an efficient representation of words and parts of words in the pre-training corpus.  The model has an embedding for each one of the subwords in its vocabulary.

We do not typically interact directly with the subword models but rather do so indirectly through the Tokenizer object.

Let's try the BERT base cased tokenizer.  It uses a wordpiece subword model. 

In [33]:
#wordpiece
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

print(f'The vocabulary size is {tokenizer.vocab_size}')

The vocabulary size is 28996


In [34]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['Don',
 "'",
 't',
 'you',
 'love',
 '[UNK]',
 'Transformers',
 '?',
 'We',
 'sure',
 'do',
 '.']

This is the same tokenizer code but instead it is loaded with the multilingual model version. Note that it contains many more tokens than BERT base because it has to be able to deal with multiple kinds of symbols.

In [35]:
#wordpiece
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
print(f'The vocabulary size is {tokenizer.vocab_size}')

The vocabulary size is 105879


In [36]:
tokenizer.tokenize("你不喜欢🤗变形金刚吗？ 我们肯定会。")

['你',
 '不',
 '喜',
 '欢',
 '[UNK]',
 '变',
 '形',
 '金',
 '刚',
 '吗',
 '？',
 '我',
 '们',
 '肯',
 '定',
 '会',
 '。']

Let's put that first English sentence through the multilingual tokenizer.  It produces the same subwords for English even though it can also handle other languages as shown by it's much lager vocabulary size.

In [37]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['don',
 "'",
 't',
 'you',
 'love',
 '[UNK]',
 'transformers',
 '?',
 'we',
 'sure',
 'do',
 '.']

T5 uses the sentencepiece subword model.  Here we'll use the tokenizer for the multilingual version of T5 called mt5.  Notice the vocabulary size.

In [38]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
print(f'The vocabulary size is {tokenizer.vocab_size}')

The vocabulary size is 250100


In [39]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['▁Don',
 "'",
 't',
 '▁you',
 '▁love',
 '▁',
 '🤗',
 '▁',
 'Transformers',
 '?',
 '▁We',
 '▁sure',
 '▁do',
 '.']

The sentencepiece subword model includes a marker to indicate if a subword is at the begining of a word and thus, in English, is preceeded by a space.  This means that with sentence piece it is possible to accurately reconstruct the sentence because we explicitly identify the word boundaries.


Finally, let's look at **GPT2** which uses the BytePair Encoding subword model.  Its output will be completely different.

In [40]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(f'The vocabulary size is {tokenizer.vocab_size}')

The vocabulary size is 50257


In [41]:
tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")

['Don',
 "'t",
 'Ġyou',
 'Ġlove',
 'ĠðŁ',
 '¤',
 'Ĺ',
 'ĠTransformers',
 '?',
 'ĠWe',
 'Ġsure',
 'Ġdo',
 '.']

[Return to Top](#returnToTop)  
<a id = 'shakespeare'></a>

## 6. Shakespeare-to-Modern English Translation with Seq2Seq Transformers

In this example, we'll build a sequence-to-sequence Transformer based model, which
we'll train on a Shakespearian English to Modern English machine translation task.

We'll need to:

- Create subword models for both Shakespearean English and modern English.
- Use a `TransformerEncoder` layer, a `TransformerDecoder` layer,
and a `PositionalEmbedding` layer.
- Prepare data as sentence pairs for training a sequence-to-sequence machine translation model.
- Use the trained model to generate translations.

How does this differ from using T5 or M2M100?  This model is not pre-trained so it knows nothing about language when we start to train it.


Let's take advantage of a new set of Keras functionality -- [KerasNLP](https://keras.io/keras_nlp/).  This library gives you access to a variety of layers as well as pretrained models that you can use.

In [42]:
!pip install -q git+https://github.com/keras-team/keras-nlp.git --upgrade

  Preparing metadata (setup.py) ... done


In [43]:
import keras_nlp

In [44]:
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

Let's define a set of hyperparameters that we can use to configure this model.

In [45]:
BATCH_SIZE = 64
EPOCHS = 15  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 40

#The size of our source and target language vocabularies
ORG_VOCAB_SIZE = 15000
MOD_VOCAB_SIZE = 15000

#define some hyperparameter values for our transformers
EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

### 6.1 Downloading the data

The data includes aligned sentences from a number of plays by William Shakespeare.  The data was copied from this repo --[https://github.com/cocoxu/Shakespeare](https://github.com/cocoxu/Shakespeare) -- and consolidated into one file for easier handling.

You will to grab a copy from our git repo and import it to your Google drive.  From there you'll be able to easily load it in to a Colab notebook.

In [46]:
#This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
#Modify this path to the appropriate location in your Drive
text_file = 'drive/MyDrive/data/train_plays-org-mod.txt'

### 6.2 Parsing the data

Each line contains a Shakespearean sentence and its corresponding modern English translation.
The Shakesperean sentence is the *source sequence* and modern English one is the *target sequence*.


In [48]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    org, mod = line.split("\t")
    org = org.lower()
    mod = mod.lower()
    text_pairs.append((org, mod))

In [49]:
#look at some examples
for _ in range(5):
    print(random.choice(text_pairs))

('more will i do— though all that i can do is nothing worth, since that my penitence comes after all, imploring pardon.', 'i will do more, though nothing i can do is any good since this remorse comes after the crime, asking for pardon.')
('he hath done good service, lady, in these wars.', 'signior benedick served well in the war, my lady.')
('have you no more to say?', 'do you have anything else to tell me?')
('is this face hero’s?', 'is this face hero’s?')
('you shall, mark antony.', 'you shall do it, mark antony.')


In [50]:
#Let's create some splits
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

19088 total pairs
13362 training pairs
2863 validation pairs
2863 test pairs


Note we have roughly 13,000 sentence pairs for training from scratch.  How does this compare with the number of sentences in other sentence pair corpora we've seen?

### 6.3 Build the wordpiece subword models

We'll define two tokenizers with subword models - one for the source language (Shakespearean English), and the other for the target language (Modern English). We'll be using `keras_nlp.tokenizers.WordPieceTokenizer` to tokenize the text. `keras_nlp.tokenizers.WordPieceTokenizer takes our training text and generates a WordPiece vocabulary of the size we specify. It also has functions for tokenizing the text, and detokenizing sequences of tokens.

As part of creating two tokenizers, we first need to train the subword models on the datasets we have. The WordPiece tokenization algorithm is the subword tokenization algorithm that we'll use here. KerasNLP makes it very simple to train WordPiece on a corpus with the  `keras_nlp.tokenizers.compute_word_piece_vocabulary` utility.

In [51]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf.data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab

We'll add four reserved tokensto our vocabulary:

- `"[PAD]"` - Padding token to append to the input sequence length when the input sequence length is shorter than the maximum sequence length.
- `"[UNK]"` - Unknown token.
- `"[START]"` - Token that marks the start of the sequence.
- `"[END]"` - Token that marks the end of the sequence.

In [52]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

# Shakespearean English
org_samples = [text_pair[0] for text_pair in train_pairs]
org_vocab = train_word_piece(org_samples, ORG_VOCAB_SIZE, reserved_tokens)

# modern English
mod_samples = [text_pair[1] for text_pair in train_pairs]
mod_vocab = train_word_piece(mod_samples, MOD_VOCAB_SIZE, reserved_tokens)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [53]:
print("Shakespearean English Tokens: ", org_vocab[100:110])
print("Modern English Tokens: ", mod_vocab[100:110])

Shakespearean English Tokens:  ['thy', 'thee', 'now', 'our', 'come', 'am', 'sir', '##d', '##ly', 'well']
Modern English Tokens:  ['go', 'can', 'now', 'she', 'was', 'they', 'come', 'there', '##ly', 'let']


In [54]:
org_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=org_vocab, lowercase=True
)
mod_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=mod_vocab, lowercase=True
)

In [55]:
org_input_ex = text_pairs[0][0]
org_tokens_ex = org_tokenizer.tokenize(org_input_ex)
print("Shakesperean English sentence: ", org_input_ex)
print("Tokens: ", org_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    org_tokenizer.detokenize(org_tokens_ex),
)

print()

mod_input_ex = text_pairs[0][1]
mod_tokens_ex = mod_tokenizer.tokenize(mod_input_ex)
print("Modern English sentence: ", mod_input_ex)
print("Tokens: ", mod_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    mod_tokenizer.detokenize(mod_tokens_ex),
)

Shakesperean English sentence:  a gentle scroll.—fair lady, by your leave: i come by note, to give and to receive.
Tokens:  tf.Tensor(
[  19  382   37 2145 1635  540   11   53  214  193    9   96   72  212
   14   27  104   96  617    9   60  158   59   60 1275   11], shape=(26,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'a gentle scroll . \xe2\x80\x94 fair lady , by your leave : i come by note , to give and to receive .', shape=(), dtype=string)

Modern English sentence:  beautiful lady, excuse me; i come off to the side, to give and to receive.
Tokens:  tf.Tensor(
[ 395  207    9  923   66   15   27  106  258   58   56  739    9   58
  156   57   58 1664   11], shape=(19,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'beautiful lady , excuse me ; i come off to the side , to give and to receive .', shape=(), dtype=string)


### 6.4 Format Datasets

In [56]:
def preprocess_batch(org, mod):
    batch_size = tf.shape(mod)[0]

    org = org_tokenizer(org)
    mod = mod_tokenizer(mod)

    # Pad `shakespearean` to `MAX_SEQUENCE_LENGTH`.
    org_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=org_tokenizer.token_to_id("[PAD]"),
    )
    org = org_start_end_packer(org)

    # Add special tokens (`"[START]"` and `"[END]"`) to `modern english` and pad it as well.
    mod_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=mod_tokenizer.token_to_id("[START]"),
        end_value=mod_tokenizer.token_to_id("[END]"),
        pad_value=mod_tokenizer.token_to_id("[PAD]"),
    )
    mod = mod_start_end_packer(mod)

    return (
        {
            "encoder_inputs": org,
            "decoder_inputs": mod[:, :-1],
        },
        mod[:, 1:],
    )


def make_dataset(pairs):
    org_texts, mod_texts = zip(*pairs)
    org_texts = list(org_texts)
    mod_texts = list(mod_texts)
    dataset = tf.data.Dataset.from_tensor_slices((org_texts, mod_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()

#make the training data
train_ds = make_dataset(train_pairs)

#make the validation data
val_ds = make_dataset(val_pairs)

In [57]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 40)
inputs["decoder_inputs"].shape: (64, 40)
targets.shape: (64, 40)


### 6.5 Define the model


We first need an embedding layer with a vector for every token in our input sequence and a positional
embedding layer which encodes the word order in the sequence. The convention is
to add these two embeddings. KerasNLP has a `keras_nlp.layers.TokenAndPositionEmbedding `
layer which does all of the above steps for us.

The sequence-to-sequence Transformer consists of a `keras_nlp.layers.TransformerEncoder`
layer and a `keras_nlp.layers.TransformerDecoder` layer working together.

The `keras_nlp.layers.TransformerEncoder` produces a representation of the input. The `keras_nlp.layers.TransformerDecoder` predicts the next words in the target sequence (N+1 and beyond).


In [58]:
# Encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=ORG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


In [59]:
# Decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=MOD_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(MOD_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

In [60]:
#connect the encoder and decoder together in sequence
seq2seq = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="s2sTransformer",
)

In [61]:
seq2seq.summary()

Model: "s2sTransformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 token_and_position_embedding (  (None, None, 256)   3850240     ['encoder_inputs[0][0]']         
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   1315072     ['token_and_position

In [62]:
seq2seq.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

### 6.6 Train the model

In [63]:
seq2seq.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/15
209/209 [==============================] - 49s 190ms/step - loss: 5.2528 - accuracy: 0.2040 - val_loss: 4.5082 - val_accuracy: 0.2516
Epoch 2/15
209/209 [==============================] - 16s 77ms/step - loss: 4.4278 - accuracy: 0.2643 - val_loss: 4.1734 - val_accuracy: 0.2896
Epoch 3/15
209/209 [==============================] - 16s 77ms/step - loss: 4.0777 - accuracy: 0.3138 - val_loss: 3.8394 - val_accuracy: 0.3490
Epoch 4/15
209/209 [==============================] - 15s 74ms/step - loss: 3.7776 - accuracy: 0.3618 - val_loss: 3.6164 - val_accuracy: 0.3825
Epoch 5/15
209/209 [==============================] - 16s 74ms/step - loss: 3.5418 - accuracy: 0.3938 - val_loss: 3.4984 - val_accuracy: 0.4021
Epoch 6/15
209/209 [==============================] - 16s 75ms/step - loss: 3.3407 - accuracy: 0.4202 - val_loss: 3.3575 - val_accuracy: 0.4238
Epoch 7/15
209/209 [==============================] - 16s 75ms/step - loss: 3.1646 - accuracy: 0.4440 - val_loss: 3.3347 - val_accuracy

### 6.7 Generate and examine some test sentences

Finally, let's demonstrate how to translate brand new sentences.
We simply feed into the model the vectorized Elizabethean English sentence
as well as the target token `"[start]"`, then we repeatedly generated the next token, until
we hit the token `"[end]"`.

In [66]:
def decode_sequences(input_sentences):
    batch_size = tf.shape(input_sentences)[0]

    # Tokenize the encoder input.
    encoder_input_tokens = org_tokenizer(input_sentences).to_tensor(
        shape=(None, MAX_SEQUENCE_LENGTH)
    )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def token_probability_fn(decoder_input_tokens):
        return seq2seq([encoder_input_tokens, decoder_input_tokens])[:, -1, :]

    # Set the prompt to the "[START]" token.
    prompt = tf.fill((batch_size, 1), mod_tokenizer.token_to_id("[START]"))

    generated_tokens = keras_nlp.utils.top_p_search(
        token_probability_fn,
        prompt,
        p=0.1,
        max_length=40,
        end_token_id=mod_tokenizer.token_to_id("[END]"),
    )
    generated_sentences = mod_tokenizer.detokenize(generated_tokens)
    return generated_sentences


test_org_texts = [pair[0] for pair in test_pairs]
for i in range(5):
    input_sentence = random.choice(test_org_texts)
    translated = decode_sequences(tf.constant([input_sentence]))
    translated = translated.numpy()[0].decode("utf-8")
    translated = (
        translated.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )
    print(f"** Example {i} **")
    print(input_sentence)
    print(translated)
    print()

** Example 0 **
there’s strange news come, sir.
there ’ s strange news .

** Example 1 **
to anger him, we’ll have the bear again, and we will fool him black and blue.
we ’ ll be ar , we ’ ll get out of the fool and black as black and black and some such an an an a fool .

** Example 2 **
the single and peculiar life is bound with all the strength and armour of the mind to keep itself from noyance; but much more that spirit upon whose weal depends and rests the lives of many.
the more clirteclim , and dulimate is a comp over the convernchieved to keep us to keep that we ’ s

** Example 3 **
what’s his fault?
what ’ s his fault ?

** Example 4 **
the table's full.
the table ' s paper .



**QUESTION 2**: What things could we do to improve the output?
* add more sentence pairs
* ensure a good distribution over all the sentence lengths
* add an encoder layer and a decoder layer
* ???

[Return to Top](#returnToTop)  
<a id = 'answers'></a>

## ANSWERS

1.  The T5 model doesn't have the token type ids that BERT uses to identify different segments.

2.  The first two suggestions -- more sentence pairs and better balance on length are a good start.  More and better data typically lead to improved performance.  We might also look into separtely "pre-training" our encoder and decoder with their own language models.  We could then use those as pre-trained models as a foundation on which we train our connected encoder and decoder.  We could also look in to using back translation to augment our existing sentence pairs. 
